# Optimisation Run Flexibility
It is possible to change the configuration of the algorithm part way through the optimization process, or even to switch algorithms completely.
This allows an optimization process to be configured to be more exploritative early on, to explore the whole design space, then later to be more exploitative, to home in in exact optimal solutions.
Doing so requires using [Platypus](https://platypus.readthedocs.io/en/latest/) algorithms directly, instead of the algorithm wrappers provided through the BESOS optimizer module.

In [ ]:
import platypus
from besos import eppy_funcs as ef, optimizer
from besos.evaluator import EvaluatorEP
from besos.parameters import expand_plist
from besos.problem import EPProblem

First we create an example problem, see [here](BuildingOptimization.ipynb) for details.

In [ ]:
idf = ef.get_idf()
parameters = expand_plist(
    {
        "NonRes Fixed Assembly Window": {
            "UFactor": (0.1, 5),
            "Solar Heat Gain Coefficient": (0.01, 0.99),
        },
        "Mass NonRes Wall Insulation": {"Thickness": (0.01, 0.09)},
    }
)
objectives = ["Electricity:Facility", "Gas:Facility"]
problem = EPProblem(parameters, objectives)
evaluator = EvaluatorEP(problem, idf)

Next we set up NSGA-II as the first algorithm, a good general purpose multi-objective genetic algorithm.
The `to_platypus` shortcut converts the Evaluator object to a `platypus.Problem` object.

In [ ]:
platypus_problem = evaluator.to_platypus()
algorithm = platypus.NSGAII(problem=platypus_problem)

Now we can run the algorithm for a lot of generations, and pause it at some point.
Use the **stop button** at the top of the notebook to interrupt the following cell.
Note: The output from the next cells will vary from run to run, due to the randomness of the underlying algorithm as well as the amount of time this cell is run for.

In [ ]:
try:
    algorithm.run(10)
except KeyboardInterrupt:
    print("Algorithm interrupted")
algorithm.population[:5]

Now we want to continue from where the first algorithm left off, running `EpsMOEA` for 10 evaluations.
In order to make the population carry over, we use the `InjectedPopulation` generator, then run the second algorithm.

If we had let the first algorithm finish, we could use `algorithm.result` instead of `algorithm.population` to use the solutions found by the first algorithm as a starting point for the next.

In [ ]:
generator = platypus.InjectedPopulation(algorithm.population)
alg2 = platypus.EpsMOEA(
    problem=platypus_problem, generator=generator, epsilons=3, population_size=10
)
alg2.run(10)

Now we convert the solutions to a dataframe using the BESOS helper function and display them.

In [ ]:
optimizer.solutions_to_df(alg2.result, problem, parts=["inputs", "outputs"])